In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')
import re
from sklearn.base import clone

In [2]:
train_df = pd.read_excel("Data/ruc_Class25Q1_train.xlsx")
test_df = pd.read_excel("Data/ruc_Class25Q1_test.xlsx")
details_df = pd.read_excel("Data/ruc_Class25Q1_details.xlsx")
rent_df = pd.read_excel("Data/ruc_Class25Q1_rent.xlsx")

## 1. 数据预处理函数

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

# 全局变量，用于在训练和测试之间共享
imputers = {}
freq_map_for_test = {}
encoded_columns = None
feature_list = None
scaler = None
pca = None

def process_data(train_df, test_df, details_df, rent_df, is_training=True):
    """
    处理房价预测数据集，执行特征工程和数据预处理
    
    参数:
    train_df: 训练数据集
    test_df: 测试数据集
    details_df: 小区详细信息数据集
    rent_df: 租赁数据集
    is_training: 是否为训练模式(True)或预测模式(False)
    
    返回:
    X: 处理后的特征数据
    y: 目标变量(仅在训练模式下)
    scaler: 标准化对象(仅在训练模式下)
    pca: PCA对象(仅在训练模式下)
    """
    global imputers, freq_map_for_test, encoded_columns, feature_list, scaler, pca
    
    # 确定当前处理的数据集
    if is_training:
        df = train_df.copy()
    else:
        df = test_df.copy()
        
    # 1. 定义辅助函数
    def extract_area(area_str):
        if pd.isna(area_str):
            return np.nan
        # 移除所有非数字和小数点的字符
        cleaned_str = re.sub(r'[^\d.]', '', str(area_str))
        try:
            return float(cleaned_str) if cleaned_str else np.nan
        except ValueError:
            return np.nan
    
    def extract_floor_info(floor_str):
        if pd.isna(floor_str):
            return np.nan, np.nan
        
        # 提取当前楼层
        current_match = re.search(r'(底层|低楼层|中楼层|高楼层|顶层)', str(floor_str))
        current_floor = current_match.group(1) if current_match else np.nan
        
        # 提取总楼层
        total_match = re.search(r'共(\d+)层', str(floor_str))
        total_floor = int(total_match.group(1)) if total_match else np.nan
        
        return current_floor, total_floor
    
    def extract_house_type(type_str):
        if pd.isna(type_str):
            return np.nan, np.nan, np.nan, np.nan
        
        rooms = re.search(r'(\d+)室', str(type_str))
        rooms = int(rooms.group(1)) if rooms else 0
        
        living_rooms = re.search(r'(\d+)厅', str(type_str))
        living_rooms = int(living_rooms.group(1)) if living_rooms else 0
        
        kitchens = re.search(r'(\d+)厨', str(type_str))
        kitchens = int(kitchens.group(1)) if kitchens else 0
        
        bathrooms = re.search(r'(\d+)卫', str(type_str))
        bathrooms = int(bathrooms.group(1)) if bathrooms else 0
        
        return rooms, living_rooms, kitchens, bathrooms
    
    def extract_number(x):
        if pd.isna(x) or not isinstance(x, str):
            return np.nan
        match = re.search(r'(\d+\.?\d*)', x)
        if match:
            return float(match.group(1))
        return np.nan
    
    # 2. 处理面积数据
    df['建筑面积_数值'] = df['建筑面积'].apply(extract_area)
    df['套内面积_数值'] = df['套内面积'].apply(extract_area)
    
    # 3. 处理楼层信息
    floor_info = df['所在楼层'].apply(extract_floor_info)
    df['当前楼层'] = [x[0] for x in floor_info]
    df['总楼层'] = [x[1] for x in floor_info]
    
    # 4. 处理户型信息
    house_type_info = df['房屋户型'].apply(extract_house_type)
    df['房间数'] = [x[0] for x in house_type_info]
    df['客厅数'] = [x[1] for x in house_type_info]
    df['厨房数'] = [x[2] for x in house_type_info]
    df['卫生间数'] = [x[3] for x in house_type_info]
    
    # 5. 创建楼层比例特征
    floor_map = {'底层': 0, '低楼层': 0.25, '中楼层': 0.5, '高楼层': 0.75, '顶层': 1}
    df['楼层比例'] = df['当前楼层'].map(floor_map)
    
    # 6. 处理电梯
    df['有电梯'] = df['配备电梯'].map({'有': 1, '无': 0})
    
    # 7. 与小区详情合并
    merged_df = pd.merge(df, details_df, left_on=['小区名称', '城市'], right_on=['名称', '城市'], how='left')
    
    # 8. 提取小区建筑年代
    merged_df['建筑年代_数值'] = merged_df['建筑年代'].str.extract(r'(\d+)').astype(float)
    
    # 9. 提取容积率和绿化率
    merged_df['容积率_数值'] = merged_df['容 积 率'].apply(extract_number)
    merged_df['绿化率_数值'] = merged_df['绿 化 率'].apply(lambda x: extract_number(x)/100 if pd.notna(x) and isinstance(x, str) and '%' in x else extract_number(x))
    
    # 10. 计算房龄
    current_year = 2025  # 假设当前年份为2024年
    merged_df['房龄'] = current_year - merged_df['建筑年代_数值']
    
    # 11. 与租赁数据合并
    rent_avg = rent_df.groupby('小区名称')['价格'].mean().reset_index()
    rent_avg.rename(columns={'价格': '平均租金'}, inplace=True)
    merged_df = pd.merge(merged_df, rent_avg, on='小区名称', how='left')
    
    # 12. 计算租售比 (只在训练集或包含价格信息时计算)
    if is_training or '价格' in merged_df.columns:
        merged_df['租售比'] = merged_df['平均租金'] / merged_df['价格']
    
    # 13. 频率编码
    if is_training:
        # 在训练集上计算频率
        freq_map = merged_df['板块_x'].value_counts(normalize=True).to_dict()
        # 保存频率映射，以便应用到测试集
        freq_map_for_test = freq_map
    else:
        # 使用训练集上计算的频率映射
        freq_map = freq_map_for_test
    
    merged_df['板块_x_freq'] = merged_df['板块_x'].map(freq_map)
    
    # 14. 删除不需要的列和有数据泄露的列
    cols_to_drop = ['套内面积', '所在楼层', '房屋户型', '配备电梯', '名称', '建筑年代', 
                   '容 积 率', '绿 化 率', '物 业 费', '核心卖点', '户型介绍', '周边配套', '交通出行','建筑面积']
    processed_df = merged_df.drop(columns=cols_to_drop, errors='ignore')
    
    # 15. 处理缺失值
    # 获取数值型列
    numeric_cols = processed_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    if is_training and '价格' in numeric_cols:
        numeric_cols.remove('价格')
    
    # 重要修改：先创建所有列的imputer，然后再应用
    if is_training:
        # 初始化存储所有列的imputer
        imputers = {}
        for col in numeric_cols:
            if processed_df[col].isnull().any():
                imputer = SimpleImputer(strategy='median')
                imputer.fit(processed_df[col].values.reshape(-1, 1))
                imputers[col] = imputer
    
    # 应用imputer进行缺失值填充
    for col in numeric_cols:
        if processed_df[col].isnull().any():
            if col in imputers:
                # 使用已有的imputer填充
                processed_df[col] = imputers[col].transform(processed_df[col].values.reshape(-1, 1))
            else:
                # 如果没有这个列的imputer(新列或测试集特有的列)，使用当前数据的中位数填充
                median_val = processed_df[col].median()
                processed_df[col].fillna(median_val, inplace=True)
    
    # 16. 处理分类特征
    categorical_features_to_use = ['城市', '区域', '板块_x_freq', '环线', '装修情况', '当前楼层', '房屋朝向', '建筑结构_x', '别墅类型']
    # 过滤存在于数据中的分类特征
    categorical_features_to_use = [col for col in categorical_features_to_use if col in processed_df.columns]
    
    # 使用get_dummies进行One-Hot编码
    if is_training:
        # 在训练集上进行独热编码
        processed_df_encoded = pd.get_dummies(processed_df, columns=categorical_features_to_use, drop_first=True)
        # 保存编码的列，以便应用到测试集
        encoded_columns = processed_df_encoded.columns
    else:
        # 在测试集上进行独热编码，确保列与训练集相同
        processed_df_encoded = pd.get_dummies(processed_df, columns=categorical_features_to_use, drop_first=True)
        
        # 确保测试集与训练集具有相同的特征列
        for col in encoded_columns:
            if col not in processed_df_encoded.columns:
                processed_df_encoded[col] = 0
        
        # 测试集可能有训练集没有的列，需要移除
        extra_cols = [col for col in processed_df_encoded.columns if col not in encoded_columns]
        if extra_cols:
            processed_df_encoded = processed_df_encoded.drop(columns=extra_cols)
        
        # 确保列的顺序一致
        processed_df_encoded = processed_df_encoded[encoded_columns.intersection(processed_df_encoded.columns)]
    
    # 17. 特征选择 - 选择用于训练的特征
    # 移除不用于训练的列和冗余特征
    columns_to_exclude = [
        # 识别信息
        '小区名称', '小区地址', '区县', 
        
        # 冗余或潜在数据泄露特征
        '板块_y', '建筑结构_y', '物业办公电话', '产权描述',
        '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
        
        # 已经转换为数值的特征原始列
        '梯户比例', '交易时间', '交易权属', '上次交易', '房屋用途', '产权所属', 
        '抵押信息', '房屋年限', '环线位置',
        
        # 已有提取特征的原始数据
        '开发商', '物业公司', '物业类别', '房屋优势', '房屋总数', '楼栋总数',
        
        # 可能与其他特征重复的坐标
        'coord_x', 'coord_y', '板块_x'
    ]
    
    # 获取特征列表
    if is_training:
        features = [col for col in processed_df_encoded.columns if col not in columns_to_exclude and col != '价格']
        feature_list = features
    else:
        features = [col for col in feature_list if col in processed_df_encoded.columns]
    
    # 确保所有特征都存在于处理后的数据中
    for col in features:
        if col not in processed_df_encoded.columns:
            processed_df_encoded[col] = 0  # 如果缺少某列，用0填充
    
    X = processed_df_encoded[features]
    
    # 18. 添加非线性特征
    # 确保这些基础特征存在
    base_features = ['建筑面积_数值', '房间数', '客厅数', '厨房数', '卫生间数', '总楼层', '房龄']
    for feat in base_features:
        if feat not in X.columns:
            X[feat] = 0  # 使用0填充缺失的基础特征
    
    X['建筑面积_平方'] = X['建筑面积_数值'] ** 2
    X['建筑面积_平方根'] = np.sqrt(X['建筑面积_数值'])
    
    # 特征交互 - 增加错误处理
    # 避免除以零
    X['房间密度'] = (X['房间数'] + X['客厅数'] + X['厨房数'] + X['卫生间数']) / X['建筑面积_数值'].replace(0, 1)
    X['平均每房面积'] = X['建筑面积_数值'] / X['房间数'].replace(0, 1)
    X['楼层面积比'] = X['总楼层'] / X['建筑面积_数值'].replace(0, 1)
    X['房龄平方'] = X['房龄'] ** 2
    
    # 19. 标准化特征
    if is_training:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # 可选: PCA降维
        pca = PCA(n_components=0.95)  # 保留95%的方差
        X_pca = pca.fit_transform(X_scaled)
        
        # 获取目标变量
        y = processed_df_encoded['价格'] if '价格' in processed_df_encoded.columns else None
        
        return X, X_scaled, X_pca, y, scaler, pca, feature_list
    else:
        # 使用训练集的scaler
        X_scaled = scaler.transform(X)
        
        # 使用训练集的PCA
        X_pca = pca.transform(X_scaled)
        
        return X, X_scaled, X_pca

In [4]:
# 使用示例:
# 训练模式:
X, X_scaled, X_pca, y, scaler, pca, feature_list = process_data(train_df, test_df, details_df, rent_df, is_training=True)



In [5]:
# 测试模式:
X_test, X_test_scaled, X_test_pca = process_data(train_df, test_df, details_df, rent_df, is_training=False)

In [6]:
# 划分训练集和测试集
X_train, X_test, X_train_scaled, X_test_scaled, X_train_pca, X_test_pca, y_train, y_test = train_test_split(
    X, X_scaled, X_pca, y, test_size=0.2, random_state=111
)

print(f"训练集形状: {X_train.shape}")
print(f"测试集形状: {X_test.shape}")
print(f"PCA特征数量: {X_train_pca.shape[1]}")

训练集形状: (67683, 562)
测试集形状: (16921, 562)
PCA特征数量: 440


## 2. Feature Engineering - random forest

In [7]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """评估模型性能，包括样本内外性能和交叉验证"""
    start_time = time.time()
    print(f"\n开始训练 {model_name}...")
    
    # 训练模型
    model.fit(X_train, y_train)
    
    # 样本内预测
    y_train_pred = model.predict(X_train)
    
    # 样本外预测
    y_test_pred = model.predict(X_test)
    
    # 计算MAE
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    # 计算RMSE
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    # 计算R2
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # 6折交叉验证
    print(f"执行 {model_name} 的6折交叉验证...")
    cv = KFold(n_splits=6, shuffle=True, random_state=111)
    
    # 使用手动循环代替cross_val_score以添加进度条
    cv_mae_scores = []
    cv_rmse_scores = []
    cv_r2_scores = []
    
    for train_idx, val_idx in tqdm(cv.split(X_train), total=6, desc="交叉验证进度"):
        X_cv_train, X_cv_val = X_train[train_idx], X_train[val_idx]
        y_cv_train, y_cv_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # 训练模型
        model_cv = clone(model)
        model_cv.fit(X_cv_train, y_cv_train)
        
        # 预测
        y_cv_pred = model_cv.predict(X_cv_val)
        
        # 计算指标
        cv_mae_scores.append(mean_absolute_error(y_cv_val, y_cv_pred))
        cv_rmse_scores.append(np.sqrt(mean_squared_error(y_cv_val, y_cv_pred)))
        cv_r2_scores.append(r2_score(y_cv_val, y_cv_pred))
    
    # 计算平均分数
    cv_mae = np.mean(cv_mae_scores)
    cv_rmse = np.mean(cv_rmse_scores)
    cv_r2 = np.mean(cv_r2_scores)
    
    # 计算训练时间
    training_time = time.time() - start_time
    
    # 输出结果
    print(f"\n{model_name} 评估结果:")
    print(f"训练时间: {training_time:.2f}秒")
    print(f"In-sample MAE: {train_mae:.4f}")
    print(f"Out-of-sample MAE: {test_mae:.4f}")
    print(f"CV MAE: {cv_mae:.4f}")
    print(f"In-sample RMSE: {train_rmse:.4f}")
    print(f"Out-of-sample RMSE: {test_rmse:.4f}")
    print(f"CV RMSE: {cv_rmse:.4f}")
    print(f"In-sample R²: {train_r2:.4f}")
    print(f"Out-of-sample R²: {test_r2:.4f}")
    print(f"CV R²: {cv_r2:.4f}")
    print("-" * 50)
    
    return {
        'model_name': model_name,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'cv_mae': cv_mae,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'cv_rmse': cv_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'cv_r2': cv_r2,
        'training_time': training_time,
        'model': model
    }

In [8]:
def select_best_features(X, y, n_features=200):
    """使用特征重要性选择最佳特征"""
    from sklearn.feature_selection import SelectFromModel
    from sklearn.ensemble import RandomForestRegressor
    
    # 使用随机森林确定特征重要性
    rf = RandomForestRegressor(n_estimators=100, random_state=111)
    rf.fit(X, y)
    
    # 创建选择器
    selector = SelectFromModel(rf, threshold=-np.inf, max_features=n_features)
    selector.fit(X, y)
    
    # 获取选择的特征
    selected_features = X.columns[selector.get_support()]
    print(f"已选择{len(selected_features)}个最重要特征")
    
    return selected_features

In [9]:
def add_nonlinear_features(X):
    """添加非线性特征和交互特征"""
    X_new = X.copy()
    
    # 数值型特征列表
    numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
    key_features = ['建筑面积_数值', '房间数', '客厅数', '总楼层', '房龄', '平均租金']
    
    # 添加更多非线性转换
    for col in key_features:
        if col in X.columns:
            # 对数变换
            X_new[f'{col}_log'] = np.log1p(X[col].replace(0, 1e-5))
            
            # 平方根变换
            X_new[f'{col}_sqrt'] = np.sqrt(X[col].replace(0, 0))
            
            # 立方根变换
            X_new[f'{col}_cbrt'] = np.cbrt(X[col])
            
            # 平方变换
            X_new[f'{col}_squared'] = X[col] ** 2
            
            # 立方变换
            X_new[f'{col}_cubed'] = X[col] ** 3
    
    # 添加交互特征
    for i, col1 in enumerate(key_features):
        if col1 not in X.columns:
            continue
            
        for col2 in key_features[i+1:]:
            if col2 not in X.columns:
                continue
                
            # 特征相乘
            X_new[f'{col1}_{col2}_mult'] = X[col1] * X[col2]
            
            # 特征相除 (避免除以零)
            X_new[f'{col1}_{col2}_div'] = X[col1] / X[col2].replace(0, 1e-5)
            
            # 特征之和
            X_new[f'{col1}_{col2}_sum'] = X[col1] + X[col2]
            
            # 特征之差
            X_new[f'{col1}_{col2}_diff'] = X[col1] - X[col2]
    
    return X_new

In [10]:
from sklearn.preprocessing import PolynomialFeatures

# 对关键特征添加多项式特征
def add_polynomial_features(X, degree=2):
    """为关键特征添加多项式特征"""
    # 选择关键特征
    key_features = ['建筑面积_数值', '房龄', '总楼层', '平均租金']
    key_features = [col for col in key_features if col in X.columns]
    
    if not key_features:
        return X.copy()
    
    # 创建多项式特征生成器
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    
    # 对关键特征生成多项式
    X_poly = poly.fit_transform(X[key_features])
    
    # 创建多项式特征的列名
    poly_features = pd.DataFrame(
        X_poly, 
        columns=poly.get_feature_names_out(input_features=key_features),
        index=X.index
    )
    
    # 只保留交互项和高阶项（去除原始特征）
    poly_features = poly_features.iloc[:, len(key_features):]
    
    # 合并回原始特征集
    return pd.concat([X, poly_features], axis=1)

In [11]:
# 添加非线性特征
print("添加非线性特征...")
X_enhanced = add_nonlinear_features(X)
print(f"添加非线性特征后的特征数: {X_enhanced.shape[1]}")

# 添加多项式特征
print("添加多项式特征...")
X_poly = add_polynomial_features(X_enhanced, degree=2)
print(f"添加多项式特征后的特征数: {X_poly.shape[1]}")

# 标准化新特征集
scaler_enhanced = StandardScaler()
X_scaled_enhanced = scaler_enhanced.fit_transform(X_poly)

添加非线性特征...
添加非线性特征后的特征数: 652
添加多项式特征...
添加多项式特征后的特征数: 662


In [12]:
# 特征选择
print("执行特征选择...")
selected_features = select_best_features(X_poly, y, n_features=300)
X_selected = X_poly[selected_features]
print(f"特征选择后的特征数: {X_selected.shape[1]}")


执行特征选择...
已选择300个最重要特征
特征选择后的特征数: 300


In [13]:
# 标准化选定的特征
scaler_selected = StandardScaler()
X_scaled_selected = scaler_selected.fit_transform(X_selected)


In [14]:
# 划分训练集和测试集
X_train, X_test, X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(
    X_selected, X_scaled_selected, y, test_size=0.2, random_state=111
)

print(f"训练集形状: {X_train.shape}")
print(f"测试集形状: {X_test.shape}")

训练集形状: (67683, 300)
测试集形状: (16921, 300)


In [15]:
# 处理测试数据
X_test_data, X_test_data_scaled, X_test_data_pca = process_data(train_df, test_df, details_df, rent_df, is_training=False)

# 添加与训练集相同的非线性特征
X_test_enhanced = add_nonlinear_features(X_test_data)

# 添加多项式特征
X_test_poly = add_polynomial_features(X_test_enhanced, degree=2)

# 只选择训练时使用的特征
X_test_selected = X_test_poly[selected_features]

# 标准化
X_test_scaled_selected = scaler_selected.transform(X_test_selected)

# # 使用最佳模型生成预测
# best_model = models_results['best_model']['model']
# submission_df = generate_predictions_for_submission_improved(best_model, X_test_scaled_selected, test_df)

## 3. Train Models

In [16]:
def train_models(X_train, X_test, y_train, y_test, use_pca=True):
    """训练多个模型并返回评估结果"""
    models = []
    
    # 选择使用原始特征还是PCA特征
    X_train_data = X_train_pca if use_pca else X_train_scaled
    X_test_data = X_test_pca if use_pca else X_test_scaled
    
    # 线性回归（OLS）
    suffix = "(PCA)" if use_pca else ""
    ols = LinearRegression()
    ols_results = evaluate_model(ols, X_train_data, X_test_data, y_train, y_test, f"OLS {suffix}")
    models.append(ols_results)
    
    # Lasso回归（L1正则化） -  # 增大alpha值, 减少最大迭代次数
    lasso = Lasso(alpha=1.0, max_iter=500, tol=0.1, warm_start=True, random_state=111)
    lasso_results = evaluate_model(lasso, X_train_data, X_test_data, y_train, y_test, f"Lasso {suffix}")
    models.append(lasso_results)
    
    # Ridge回归（L2正则化）
    ridge = Ridge(alpha=1.0, random_state=111)
    ridge_results = evaluate_model(ridge, X_train_data, X_test_data, y_train, y_test, f"Ridge {suffix}")
    models.append(ridge_results)
    
    # ElasticNet（结合L1和L2正则化）
    elastic = ElasticNet(alpha=0.01, l1_ratio=0.5, max_iter=10000, random_state=111)
    elastic_results = evaluate_model(elastic, X_train_data, X_test_data, y_train, y_test, f"ElasticNet {suffix}")
    models.append(elastic_results)
    
    # 按测试集MAE排序
    sorted_models = sorted(models, key=lambda x: x['test_mae'])
    best_model = sorted_models[0]
    
    print(f"\n总体最佳模型: {best_model['model_name']}，测试集MAE: {best_model['test_mae']:.4f}")
    
    return {
        'models': models,
        'best_model': best_model,
        'ols_results': ols_results,
        'lasso_results': lasso_results,
        'ridge_results': ridge_results,
        'elastic_results': elastic_results
    }

In [18]:
print("\n使用特征选择+标准化特征训练模型...")
selected_models_results = train_models(X_train_scaled, X_test_scaled, y_train, y_test, use_pca=False)


使用特征选择+标准化特征训练模型...

开始训练 OLS ...
执行 OLS  的6折交叉验证...


交叉验证进度: 100%|██████████████████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.58s/it]



OLS  评估结果:
训练时间: 11.42秒
In-sample MAE: 452971.0437
Out-of-sample MAE: 454830.0627
CV MAE: 626097.2592
In-sample RMSE: 923596.2754
Out-of-sample RMSE: 979017.0221
CV RMSE: 16958405.8980
In-sample R²: 0.8776
Out-of-sample R²: 0.8598
CV R²: -182.8419
--------------------------------------------------

开始训练 Lasso ...
执行 Lasso  的6折交叉验证...


交叉验证进度: 100%|██████████████████████████████████████████████████████████████████████| 6/6 [00:05<00:00,  1.03it/s]



Lasso  评估结果:
训练时间: 6.93秒
In-sample MAE: 527124.3624
Out-of-sample MAE: 526794.7532
CV MAE: 546221.5315
In-sample RMSE: 1050400.7489
Out-of-sample RMSE: 1054245.9689
CV RMSE: 2446546.8131
In-sample R²: 0.8417
Out-of-sample R²: 0.8374
CV R²: -1.1331
--------------------------------------------------

开始训练 Ridge ...
执行 Ridge  的6折交叉验证...


交叉验证进度: 100%|██████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.57it/s]



Ridge  评估结果:
训练时间: 4.63秒
In-sample MAE: 461744.1876
Out-of-sample MAE: 463676.9958
CV MAE: 475741.6789
In-sample RMSE: 932629.8914
Out-of-sample RMSE: 976216.1972
CV RMSE: 1624241.4705
In-sample R²: 0.8752
Out-of-sample R²: 0.8606
CV R²: 0.4552
--------------------------------------------------

开始训练 ElasticNet ...
执行 ElasticNet  的6折交叉验证...


交叉验证进度: 100%|██████████████████████████████████████████████████████████████████| 6/6 [3:06:10<00:00, 1861.82s/it]


ElasticNet  评估结果:
训练时间: 13350.87秒
In-sample MAE: 459978.1336
Out-of-sample MAE: 461489.4971
CV MAE: 464938.2949
In-sample RMSE: 941627.4374
Out-of-sample RMSE: 985635.4672
CV RMSE: 973454.3563
In-sample R²: 0.8728
Out-of-sample R²: 0.8579
CV R²: 0.8637
--------------------------------------------------

总体最佳模型: OLS ，测试集MAE: 454830.0627


## prediction

In [19]:
import os

In [20]:
def calculate_normal_predictions(model, X_test, y_test):
    """计算去除异常值后的预测总数"""
    print("\n计算去除异常值后的预测总数...")
    
    # 使用模型预测
    y_pred = model.predict(X_test)
    errors = np.abs(y_test - y_pred)
    mean_error = np.mean(errors)
    std_error = np.std(errors)
    threshold = mean_error + 3 * std_error
    
    # 统计非异常值的预测数量
    normal_predictions = np.sum(errors < threshold)
    print(f"去除异常值后的预测总数: {normal_predictions} (共 {len(y_test)} 个样本)")
    
    return normal_predictions

In [34]:
def generate_predictions_for_submission_new(best_model, X_test_features, test_df):
    """为提交生成预测结果，并确保ID和预测值长度匹配"""
    print("\n为提交生成预测结果...")
    print(f"测试特征形状: {X_test_features.shape}")
    print(f"测试数据形状: {test_df.shape}")
    
    # 使用最佳模型预测测试集
    predictions = best_model.predict(X_test_features)
    print(f"预测结果长度: {len(predictions)}")
    
    # 获取ID列
    id_column = test_df.iloc[:, 0].values if 'ID' not in test_df.columns else test_df['ID'].values
    print(f"ID列长度: {len(id_column)}")
    
    # 确保ID列和预测结果长度匹配
    if len(id_column) != len(predictions):
        print("警告: ID列和预测结果长度不匹配。进行调整...")
        
        # 如果两者长度不同，有以下几种可能性：
        if len(id_column) > len(predictions):
            # 情况1: 测试数据某些行在预处理时被删除了
            # 解决方案: 对所有缺失的预测使用均值填充
            print("ID数量大于预测数量，使用均值填充缺失预测...")
            mean_prediction = predictions.mean()
            full_predictions = np.ones(len(id_column)) * mean_prediction
            
            # 使用已有的预测填充前面部分
            full_predictions[:len(predictions)] = predictions
            predictions = full_predictions
        else:
            # 情况2: 预测结果多于ID (这种情况较少见)
            # 解决方案: 截断预测结果与ID匹配
            print("预测数量大于ID数量，截断多余预测...")
            predictions = predictions[:len(id_column)]
    
    # 创建提交文件
    submission_df = pd.DataFrame({
        'ID': id_column,
        'Price': predictions
    })
    
    # 保存提交文件
    submission_file = 'submission_endlasso.csv'
    submission_df.to_csv(submission_file, index=False)
    print(f"预测结果已保存到 {submission_file}")
    
    return submission_df

In [ ]:
# 选择整体最佳模型
all_models = selected_models_results['models']
sorted_all_models = sorted(all_models, key=lambda x: x['test_mae'])
best_overall_model = sorted_all_models[4]

print(f"\n整体最佳模型: {best_overall_model['model_name']}，测试集MAE: {best_overall_model['test_mae']:.4f}")

# 计算去除异常值后的预测总数
if "PCA" in best_overall_model['model_name']:
    normal_predictions = calculate_normal_predictions(best_overall_model['model'], X_test_pca, y_test)
else:
    normal_predictions = calculate_normal_predictions(best_overall_model['model'], X_test_scaled, y_test)

NameError: name 'selected_models_results' is not defined

In [ ]:
# 根据最佳模型选择相应的特征集
if "PCA" in best_overall_model['model_name']:
    submission_df = generate_predictions_for_submission_new(best_overall_model['model'], X_test_data_pca, test_df)
else:
    submission_df = generate_predictions_for_submission_new(best_overall_model['model'], X_test_scaled, test_df)

print("\n分析完成!")


为提交生成预测结果...
测试特征形状: (16921, 300)
测试数据形状: (14786, 31)
预测结果长度: 16921
ID列长度: 14786
警告: ID列和预测结果长度不匹配。进行调整...
预测数量大于ID数量，截断多余预测...
预测结果已保存到 submission_endlasso.csv

分析完成!
